# How to plot a Fermi surface

## Using BoltzTraP v1

### PbTe mp-19717

In [1]:
from pymatgen.electronic_structure.plotter import plot_fermi_surface
from monty.os import makedirs_p
from pymatgen.symmetry.bandstructure import HighSymmKpath

In [9]:
vrun = Vasprun('boltztrap2_data/vasprun-PbTe_uniform_bs.xml')
bs = vrun.get_band_structure()
nelect = vrun.parameters['NELECT']

/usr/lib64/python3.7/site-packages/pymatgen/io/vasp/outputs.py:886: UserWarning: No POTCAR file with matching TITEL fields was found in /home/tardo/github/matgenb/notebooks/boltztrap2_data
  " was found in {}".format(os.path.abspath(p)))


#### VBM 

In [10]:
# specify here the band index you need, in this example the vbm is taken
vbm = bs.get_vbm()['band_index'][Spin.up][-1]

In [11]:
# create the directory "fermi/vbm" before
makedirs_p('fermi/vbm/')

# run the interpolation, lpfac should be between 50 and 150
BoltztrapRunner(bs=bs,nelec=nelect,lpfac=10,run_type='FERMI',band_nb=vbm,cond_band=False).run(path_dir='fermi/vbm/')

# read the output
an=BoltztrapAnalyzer.from_files("fermi/vbm/boltztrap/")

In [12]:
# prepare high symmetry points labels
st = vrun.final_structure
kpoints_dict = HighSymmKpath(st).kpath['kpoints']

In [13]:
# edit or generete a dict like this one to pass the kpoint labels and frac coords you want to plot
kpoints_dict

{'\\Gamma': array([0., 0., 0.]),
 'K': array([0.375, 0.375, 0.75 ]),
 'L': array([0.5, 0.5, 0.5]),
 'U': array([0.625, 0.25 , 0.625]),
 'W': array([0.5 , 0.25, 0.75]),
 'X': array([0.5, 0. , 0.5])}

In [15]:
an.fermi_surface_data.max()

0.005432916487335469

In [23]:
# plot the surface in an external window
# vbm is set at 0 eV, so we set an energy level of 0.05 below the vbm
plot_fermi_surface(an.fermi_surface_data,bs.structure,cbm=False,energy_levels=[-0.05], 
                   kpoints_dict=kpoints_dict, labels_scale_factor=0.1)

(<mayavi.core.scene.Scene at 0x7fcf4f4737d0>,
 <module 'mayavi.mlab' from '/home/tardo/.local/lib/python3.7/site-packages/mayavi/mlab.py'>)

![Fermi surface of vbm for PbTe](boltztrap2_data/fermi_surface_vbm_PbTe.png)

#### CBM

In [16]:
# specify here another band index you need, in this example the cbm is taken
cbm = bs.get_cbm()['band_index'][Spin.up][0]

In [17]:
# create the directory "fermi/cbm" before
makedirs_p('fermi/cbm/')

# run the interpolation, lpfac should be between 50 and 150
BoltztrapRunner(bs=bs,nelec=nelect,lpfac=10,run_type='FERMI',band_nb=cbm,cond_band=True).run(path_dir='fermi/cbm/')

# read the output
an=BoltztrapAnalyzer.from_files("fermi/cbm/boltztrap/")

In [18]:
an.fermi_surface_data.max()

-0.8262383617146892

In [20]:
# plot the surface in an external window
# cbm is at ~0.83 eV, so we set an energy level of 0.07 above the cbm

plot_fermi_surface(an.fermi_surface_data,bs.structure,cbm=True,energy_levels=[0.9], 
                   kpoints_dict=kpoints_dict, labels_scale_factor=0.1)

(<mayavi.core.scene.Scene at 0x7ffae8070350>,
 <module 'mayavi.mlab' from '/home/tardo/.local/lib/python3.7/site-packages/mayavi/mlab.py'>)

![Fermi surface of cbm for PbTe](boltztrap2_data/fermi_surface_cbm_PbTe.png)

### AlAs mp-2172
Example of how to plot multiple surfaces in the same figure

In [2]:
vrun = Vasprun('boltztrap2_data/vasprun_AlAs.xml.gz')
bs = vrun.get_band_structure()
nelect = vrun.parameters['NELECT']

/usr/lib64/python3.7/site-packages/pymatgen/io/vasp/outputs.py:886: UserWarning: No POTCAR file with matching TITEL fields was found in /home/tardo/github/matgenb/notebooks/boltztrap2_data
  " was found in {}".format(os.path.abspath(p)))


In [3]:
cbm = bs.get_cbm()['band_index'][Spin.up][0]
vb1 = bs.get_vbm()['band_index'][Spin.up][-1]
vb2 = bs.get_vbm()['band_index'][Spin.up][-2]
vb3 = bs.get_vbm()['band_index'][Spin.up][-3]

In [4]:
cbm, vb1, vb2, vb3

(4, 3, 2, 1)

In [5]:
st = vrun.final_structure
kpoints_dict = HighSymmKpath(st).kpath['kpoints']

In [6]:
kpoints_dict

{'\\Gamma': array([0., 0., 0.]),
 'K': array([0.375, 0.375, 0.75 ]),
 'L': array([0.5, 0.5, 0.5]),
 'U': array([0.625, 0.25 , 0.625]),
 'W': array([0.5 , 0.25, 0.75]),
 'X': array([0.5, 0. , 0.5])}

#### CBM

In [7]:
makedirs_p('fermi/cbm/')

# run the interpolation, lpfac should be between 50 and 150
BoltztrapRunner(bs=bs,nelec=nelect,lpfac=5,run_type='FERMI',band_nb=cbm,cond_band=True).run(path_dir='fermi/cbm/')

# read the output
an=BoltztrapAnalyzer.from_files("fermi/cbm/boltztrap/")

# plot the fermi surface and get the figure object
fig, mlab = plot_fermi_surface(an.fermi_surface_data,bs.structure,cbm=True,energy_levels=[1.55], 
                               kpoints_dict=kpoints_dict, labels_scale_factor=0.1,
                               interative=False,multiple_figure=False,color=(1,0,0))

#### First three vb

In [8]:
for vb,tf in zip((vb1,vb2,vb3),(0.2,0.4,1)):
    makedirs_p(f'fermi/vb_{vb}')
    # run the interpolation, lpfac should be between 50 and 150
    BoltztrapRunner(bs=bs,nelec=nelect,lpfac=50,run_type='FERMI',band_nb=vb,cond_band=False).run(path_dir=f'fermi/vb_{vb}/')

    # read the output
    an=BoltztrapAnalyzer.from_files(f"fermi/vb_{vb}/boltztrap/")
    
    # plot the fermi surface in the previous figure object
    fig, mlab = plot_fermi_surface(an.fermi_surface_data,bs.structure,cbm=False,energy_levels=[-0.07], 
                               kpoints_dict=kpoints_dict, labels_scale_factor=0.1,
                               interative=False, mlab_figure=fig,multiple_figure=False,transparency_factor=[tf])
mlab.show()

3
2
1


![Fermi surface of first 3 vbs and cbm for PbTe](boltztrap2_data/fermi_surface_vb_cb_AlAs.png)

# In case of a cube file from other software

In [27]:
from pymatgen.electronic_structure.boltztrap import read_cube_file

# decompress boltztrap_BZ.cube.gz before running the following
fs_data = read_cube_file('boltztrap2_data/boltztrap_BZ.cube')
st = Vasprun('boltztrap2_data/vasprun_mp-12103.xml.gz').final_structure
kpoints_dict = HighSymmKpath(st).kpath['kpoints']

/usr/lib64/python3.7/site-packages/pymatgen/io/vasp/outputs.py:886: UserWarning: No POTCAR file with matching TITEL fields was found in /home/tardo/github/matgenb/notebooks/boltztrap2_data
  " was found in {}".format(os.path.abspath(p)))


In [23]:
plot_fermi_surface(fs_data,st,cbm=True, 
                   kpoints_dict=kpoints_dict, labels_scale_factor=0.1,
                   color=(1,0,0))

Energy level set to: 1.0449469713543411 eV


(<mayavi.core.scene.Scene at 0x7ffaccb68b30>,
 <module 'mayavi.mlab' from '/home/tardo/.local/lib/python3.7/site-packages/mayavi/mlab.py'>)

![Fermi surface of cbm for N2](boltztrap2_data/fermi_surface_cbm_mp-12103.png)

## Using BoltzTraP 2 pymatgen interface

In [1]:
from pymatgen.electronic_structure.boltztrap2 import BztInterpolator, VasprunBSLoader

In [2]:
vrun = Vasprun('boltztrap2_data/vasprun-PbTe_uniform_bs.xml')
bs = vrun.get_band_structure()
nelect = vrun.parameters['NELECT']

/usr/lib64/python3.7/site-packages/pymatgen/io/vasp/outputs.py:886: UserWarning: No POTCAR file with matching TITEL fields was found in /home/tardo/github/matgenb/notebooks/boltztrap2_data
  " was found in {}".format(os.path.abspath(p)))


In [3]:
data = VasprunBSLoader(vrun)

In [4]:
bztI = BztInterpolator(data,lpfac=10)

In [5]:
bztI.eband.shape

(6, 148877)

In [6]:
from BoltzTraP2 import fermisurface

In [ ]:
# with 2:3 we get the index of the vb, mu is in Ha
fermisurface.plot_fermisurface(data,bztI.equivalences,bztI.eband[2:3,:],mu=-0.003)

![Fermi surface of vbm for PbTe](boltztrap2_data/fermi_surface_vbm_PbTe_bzt2.png)